In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
import category_encoders as ce

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
import tensorflow as tf

In [5]:
import keras
from keras.models import Model
import kerastuner

In [6]:
from sklearn.metrics import log_loss

In [7]:
from keras.layers import Dense, Input, Dropout

## Highly basic Keras notebook
1. Manually transform data: binary, simple MinMaxScaler, StandardScaler.
1. Ran another basic autokeras on this more amenable dataset; a one-hidden-layer model did ok.
1. Tune the resulting keras-from-autokeras model.    

In [8]:
tf_df = pd.read_csv('../input/lish-moa/train_features.csv',index_col='sig_id')
tts_df = pd.read_csv('../input/lish-moa/train_targets_scored.csv',index_col='sig_id')
tf_df['cp_type']=tf_df['cp_type'].astype('category')
scaler = MinMaxScaler(feature_range=(-0.5,0.5))
tf_df['cp_time']=scaler.fit_transform((np.array(tf_df['cp_time'])).reshape(-1,1))
tf_df['cp_dose']=tf_df['cp_dose'].astype('category')
transformer = StandardScaler()
tf_df.loc[:,'g-0':'c-99']=transformer.fit_transform(tf_df.loc[:,'g-0':'c-99'])

In [9]:
print(tf_df['cp_type'].unique(),tf_df['cp_dose'].unique())

['trt_cp', 'ctl_vehicle']
Categories (2, object): ['trt_cp', 'ctl_vehicle'] ['D1', 'D2']
Categories (2, object): ['D1', 'D2']


In [10]:
oenc = ce.ordinal.OrdinalEncoder()
tf_df = oenc.fit_transform(tf_df)
tf_df.sample(20)

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_84f004e1a,1,0.5,1,0.027009,0.528449,0.761455,-0.158076,-0.941164,0.081515,-0.547681,...,0.202427,-0.228693,-0.047282,0.193038,0.243196,0.268973,0.042863,0.247268,0.354238,-0.663756
id_7ed95e96b,1,0.5,2,-0.085811,0.668660,0.079990,-0.192287,0.253910,0.178092,0.531677,...,0.231170,0.546804,0.606821,0.336508,0.336642,-0.356589,0.246390,0.122883,0.669097,0.786785
id_e5ae8674c,1,0.5,1,-0.026458,0.396363,0.972036,-1.347349,0.158665,0.208193,1.602536,...,0.394084,0.536718,0.650484,0.469632,0.400452,0.637914,0.410008,0.272040,0.392335,0.285467
id_d034694bd,2,0.0,2,0.031889,-3.425037,-0.951863,-2.512622,-3.188106,-0.256549,0.604549,...,-0.081210,-0.613283,-0.111579,-0.803896,-0.096088,-0.198173,-0.285246,-0.933187,-0.005258,-0.262943
id_7c0b91323,1,-0.5,1,-0.363053,0.186231,0.150570,-0.227867,1.679783,-0.142928,0.329496,...,0.646278,-0.059434,0.796962,0.703654,0.361879,-0.382429,0.006693,0.427595,-0.157864,0.602680
id_a631edc03,1,-0.5,1,-0.615032,-0.762256,-0.432219,0.319085,1.202102,0.242534,0.331423,...,0.600338,0.468858,0.489819,0.261617,0.895140,-0.308960,0.098697,0.156577,0.181248,-0.064064
id_b41b8443a,1,0.5,2,0.212387,0.347615,0.954656,1.009512,0.376092,1.205596,0.041410,...,0.199528,-0.215375,-0.319105,0.040883,0.154428,0.257066,-0.271166,-0.492938,-2.441015,0.308267
id_8a2385bc9,2,0.0,1,0.369632,0.427261,0.522875,-1.481035,0.593519,0.438317,0.808090,...,-0.092507,0.360213,0.143708,0.552638,0.560949,0.613302,-0.272622,0.428182,0.277826,-0.164285
id_37f0b887e,1,0.5,1,0.063898,-0.297307,0.415316,0.569087,-1.254127,-1.744301,-1.136432,...,-0.307310,-0.257286,-0.196558,-0.200022,-0.061451,0.167147,-0.386232,-0.360159,0.229810,0.049795


In [11]:
tf_df['cp_type']=tf_df['cp_type']-1
tf_df['cp_dose']=tf_df['cp_dose']-1
tf_df.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0,-0.5,0,0.583933,0.804317,-0.386357,-0.739765,-0.243924,-0.740369,-1.198991,...,0.377638,0.352428,0.659876,0.499484,0.146278,0.621219,0.031745,0.347587,0.463460,0.510769
id_000779bfc,0,0.5,0,-0.124925,0.620898,0.141784,-0.022707,0.931772,0.559230,0.224552,...,0.021367,0.595227,0.491617,0.248281,0.371742,0.517798,0.298922,0.294874,0.587507,0.737704
id_000a6266a,0,0.0,0,0.272457,0.833861,1.353417,-0.166708,-0.086861,1.168288,0.153607,...,-0.127850,-0.082397,0.561309,0.247948,-0.378278,0.022296,-0.086525,0.093651,-0.511093,0.706452
id_0015fd391,0,0.0,0,-0.546995,-0.188856,-0.403446,0.470350,3.880216,-0.568667,-2.260896,...,-0.814697,-0.089447,-2.556236,-0.416627,-0.164899,-0.573436,-0.561987,-0.293593,0.045047,-0.365011
id_001626bd3,0,0.5,1,-0.411783,-0.375722,0.789552,0.642035,1.318373,-0.581300,-0.358079,...,0.236669,0.228262,0.589635,0.744707,0.490563,0.031073,0.278143,0.391377,0.049952,0.720231


In [12]:
tf_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23814 entries, id_000644bb2 to id_ffffdd77b
Columns: 875 entries, cp_type to c-99
dtypes: float64(873), int64(2)
memory usage: 159.2+ MB


In [13]:
tts_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23814 entries, id_000644bb2 to id_ffffdd77b
Columns: 206 entries, 5-alpha_reductase_inhibitor to wnt_inhibitor
dtypes: int64(206)
memory usage: 37.6+ MB


Floats and ints now, just like it needs to be.

Let's hardcode these into numpy arrays since that's technically what Keras wants.

In [14]:
tf_arr = np.array(tf_df)
tts_arr = np.array(tts_df)

Let's try the TPU. Wait, I have to have internet access turned on? Boo.

In [15]:
strategy = tf.distribute.get_strategy()
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Number of accelerators:  1


In [16]:
bce = tf.losses.BinaryCrossentropy(from_logits=True)
with strategy.scope():
    inputs = Input(shape=(875,))
    x = Dense(1024,activation='elu')(inputs)
    x = Dropout(0.2,input_shape=(1024,))(x)
    x = Dense(256,activation='elu')(x)
    x = Dropout(0.2,input_shape=(256,))(x)
    x = Dense(64,activation='elu')(x)
    x = Dropout(0.2,input_shape=(64,))(x)
    outputs = Dense(206,activation='sigmoid')(x)
    model1 = Model(inputs,outputs)
    model1.compile("adam", bce, metrics=["binary_crossentropy","accuracy"])

In [17]:
model1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 875)]             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              897024    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)               

In [18]:
n_epochs = 40
n_batch = 32
print('Starting Training')
model1.fit(tf_arr,tts_arr,epochs=n_epochs,batch_size=n_batch)
print('Finished Training')

Starting Training
Epoch 1/40
745/745 [==============================] - 3s 5ms/step - loss: 0.6981 - binary_crossentropy: 0.2224 - accuracy: 0.0058
Epoch 2/40
745/745 [==============================] - 3s 4ms/step - loss: 0.6932 - binary_crossentropy: 0.3827 - accuracy: 0.0165
Epoch 3/40
745/745 [==============================] - 2s 3ms/step - loss: 0.6931 - binary_crossentropy: 0.4336 - accuracy: 0.0188
Epoch 4/40
745/745 [==============================] - 3s 3ms/step - loss: 0.6931 - binary_crossentropy: 0.4564 - accuracy: 0.0210
Epoch 5/40
745/745 [==============================] - 2s 3ms/step - loss: 0.6931 - binary_crossentropy: 0.5092 - accuracy: 0.0223
Epoch 6/40
745/745 [==============================] - 3s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.5674 - accuracy: 0.0255
Epoch 7/40
745/745 [==============================] - 2s 3ms/step - loss: 0.6931 - binary_crossentropy: 0.6595 - accuracy: 0.0308
Epoch 8/40
745/745 [==============================] - 2s 3ms/step - loss

In [19]:
tts_pred = model1.predict(tf_arr)
log_loss(tts_arr,tts_pred)

3.7692548074519654

Wow, that's worse than my single layer model.
Now let's tune.

In [20]:
!rm -R /kaggle/working/keras_tune

rm: cannot remove '/kaggle/working/keras_tune': No such file or directory


In [21]:
def build_model(hp):
    inputs = Input(shape=(875,))
    x = Dense(
        units = hp.Choice('units 1',values=[64,256,1024]),
        activation = hp.Choice('activation function 1',values=['relu','elu'])
    )(inputs)
    x = Dropout(hp.Choice('dropout 1',values=[0.2,0.35,0.5]))(x)
    x = Dense(
        units = hp.Choice('units 2',values=[32,128,512]),
        activation = hp.Choice('activation function 2',values=['relu','elu'])
    )(x)
    x = Dropout(hp.Choice('dropout 2',values=[0.2,0.35,0.5]))(x)
    x = Dense(
        units = hp.Choice('units 3',values=[16,64,256]),
        activation = hp.Choice('activation function 3',values=['relu','elu'])
    )(x)
    x = Dropout(hp.Choice('dropout 3',values=[0.2,0.35,0.5]))(x)

    outputs = Dense(206,activation = hp.Choice('classify function',values =
                                              ['sigmoid','softmax']))(x)
    model = Model(inputs, outputs)
    # should throw in a choice of optimizer here: adam, etc.
    opt = tf.keras.optimizers.Adam(
        learning_rate=hp.Choice('learn rate',values=[0.01,0.001,0.0001]),
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False
    )
    bce = tf.losses.BinaryCrossentropy(from_logits=True)
    model.compile(optimizer=opt,loss=bce,metrics=[bce,'accuracy'])
    return model

In [22]:
# let's try this
with strategy.scope():
    tuner = kerastuner.tuners.Hyperband(
            build_model,
            objective='binary_crossentropy',
            max_epochs=18,
            executions_per_trial=2,
            directory='keras_tune'
    )

In [23]:
print('Starting Tuning')
X_train, X_val, y_train, y_val = train_test_split(tf_arr,tts_arr,test_size=0.2,random_state=84)
tuner.search(X_train,y_train,validation_data=(X_val,y_val))
print('Finished Tuning')

Starting Tuning
Epoch 1/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6954 - binary_crossentropy: 0.6954 - accuracy: 0.0351 - val_loss: 0.6952 - val_binary_crossentropy: 0.6952 - val_accuracy: 0.0775
Epoch 2/2
596/596 [==============================] - 3s 4ms/step - loss: 0.6952 - binary_crossentropy: 0.6952 - accuracy: 0.0823 - val_loss: 0.6951 - val_binary_crossentropy: 0.6951 - val_accuracy: 0.0848
Epoch 1/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6954 - binary_crossentropy: 0.6954 - accuracy: 0.0354 - val_loss: 0.6952 - val_binary_crossentropy: 0.6952 - val_accuracy: 0.0655
Epoch 2/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6952 - binary_crossentropy: 0.6952 - accuracy: 0.0617 - val_loss: 0.6952 - val_binary_crossentropy: 0.6952 - val_accuracy: 0.0741


Epoch 1/2
596/596 [==============================] - 2s 4ms/step - loss: 0.7131 - binary_crossentropy: 0.7131 - accuracy: 0.0036 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0044
Epoch 2/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6938 - binary_crossentropy: 0.6938 - accuracy: 0.0117 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0052
Epoch 1/2
596/596 [==============================] - 2s 4ms/step - loss: 0.7131 - binary_crossentropy: 0.7131 - accuracy: 0.0067 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0019
Epoch 2/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6938 - binary_crossentropy: 0.6938 - accuracy: 0.0220 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0418


Epoch 1/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6955 - binary_crossentropy: 0.6955 - accuracy: 0.0133 - val_loss: 0.6954 - val_binary_crossentropy: 0.6954 - val_accuracy: 0.0342
Epoch 2/2
596/596 [==============================] - 2s 3ms/step - loss: 0.6953 - binary_crossentropy: 0.6953 - accuracy: 0.0457 - val_loss: 0.6953 - val_binary_crossentropy: 0.6953 - val_accuracy: 0.0586
Epoch 1/2
596/596 [==============================] - 3s 5ms/step - loss: 0.6955 - binary_crossentropy: 0.6955 - accuracy: 0.0132 - val_loss: 0.6954 - val_binary_crossentropy: 0.6954 - val_accuracy: 0.0178
Epoch 2/2
596/596 [==============================] - 4s 7ms/step - loss: 0.6954 - binary_crossentropy: 0.6954 - accuracy: 0.0422 - val_loss: 0.6953 - val_binary_crossentropy: 0.6953 - val_accuracy: 0.0592


Epoch 1/2
596/596 [==============================] - 3s 5ms/step - loss: 0.6939 - binary_crossentropy: 0.6939 - accuracy: 0.3877 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3953
Epoch 2/2
596/596 [==============================] - 2s 3ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3936 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3953
Epoch 1/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6940 - binary_crossentropy: 0.6940 - accuracy: 0.3872 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3953
Epoch 2/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3932 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3953


Epoch 1/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6956 - binary_crossentropy: 0.6956 - accuracy: 0.0134 - val_loss: 0.6955 - val_binary_crossentropy: 0.6955 - val_accuracy: 6.2986e-04
Epoch 2/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6956 - binary_crossentropy: 0.6956 - accuracy: 0.0141 - val_loss: 0.6956 - val_binary_crossentropy: 0.6956 - val_accuracy: 6.2986e-04
Epoch 1/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6956 - binary_crossentropy: 0.6956 - accuracy: 0.0153 - val_loss: 0.6955 - val_binary_crossentropy: 0.6955 - val_accuracy: 6.2986e-04
Epoch 2/2
596/596 [==============================] - 2s 4ms/step - loss: 0.6956 - binary_crossentropy: 0.6956 - accuracy: 0.0155 - val_loss: 0.6955 - val_binary_crossentropy: 0.6955 - val_accuracy: 0.0311


...

Epoch 1/18
596/596 [==============================] - 3s 4ms/step - loss: 0.7000 - binary_crossentropy: 0.7000 - accuracy: 0.0092 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0334
Epoch 2/18
596/596 [==============================] - 3s 5ms/step - loss: 0.6932 - binary_crossentropy: 0.6932 - accuracy: 0.0109 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0655
Epoch 3/18
596/596 [==============================] - 3s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.0163 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0831
Epoch 4/18
596/596 [==============================] - 3s 5ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.0187 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.1436
Epoch 5/18
596/596 [==============================] - 3s 5ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.0204 - val_loss: 0.6931 - val_binary_crossentropy: 

Epoch 1/18
596/596 [==============================] - 3s 5ms/step - loss: 0.6981 - binary_crossentropy: 0.6981 - accuracy: 0.0068 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0134
Epoch 2/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.0056 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0059
Epoch 3/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.0056 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0042
Epoch 4/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.0052 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.0034
Epoch 5/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.0044 - val_loss: 0.6931 - val_binary_crossentropy: 

Epoch 1/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6946 - binary_crossentropy: 0.6946 - accuracy: 0.3623 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3802
Epoch 2/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3711 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3802
Epoch 3/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3770 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3890
Epoch 4/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3826 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3890
Epoch 5/18
596/596 [==============================] - 3s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3827 - val_loss: 0.6931 - val_binary_crossentropy: 

Epoch 1/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6938 - binary_crossentropy: 0.6938 - accuracy: 0.3892 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3953
Epoch 2/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3937 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3953
Epoch 3/18
596/596 [==============================] - 3s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3937 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3953
Epoch 4/18
596/596 [==============================] - 3s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3937 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3953
Epoch 5/18
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3937 - val_loss: 0.6931 - val_binary_crossentropy: 

Finished Tuning


Chollet put this reasonably comprehensible notebook up on Kaggle: https://www.kaggle.com/fchollet/keras-kerastuner-best-practices/ Let's try the next steps from there.

In [24]:
best_hp = tuner.get_best_hyperparameters()[0]
print(best_hp.values)

{'units 1': 64, 'activation function 1': 'relu', 'dropout 1': 0.2, 'units 2': 512, 'activation function 2': 'elu', 'dropout 2': 0.2, 'units 3': 256, 'activation function 3': 'elu', 'dropout 3': 0.5, 'classify function': 'sigmoid', 'learn rate': 0.01, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}


In [25]:
model2 = build_model(best_hp)
history = model2.fit(tf_arr, tts_arr, validation_split=0.2, epochs=50)

Epoch 1/50
596/596 [==============================] - 2s 4ms/step - loss: 0.6940 - binary_crossentropy: 0.6939 - accuracy: 0.3883 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3932
Epoch 2/50
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3942 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3932
Epoch 3/50
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3943 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3932
Epoch 4/50
596/596 [==============================] - 2s 4ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3943 - val_loss: 0.6931 - val_binary_crossentropy: 0.6931 - val_accuracy: 0.3932
Epoch 5/50
596/596 [==============================] - 2s 3ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3942 - val_loss: 0.6931 - val_binary_crossentropy: 

In [26]:
model2.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 875)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                56064     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               33280     
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)              

In [27]:
val_logloss_per_epoch = history.history['val_binary_crossentropy']
best_epoch = val_logloss_per_epoch.index(max(val_logloss_per_epoch)) + 1
best_epoch

1

In [28]:
model2a = build_model(best_hp)
model2a.fit(tf_arr, tts_arr, epochs=best_epoch)

745/745 [==============================] - 3s 3ms/step - loss: 0.6938 - binary_crossentropy: 0.6938 - accuracy: 0.3891


In [29]:
model2a.evaluate(tf_arr,tts_arr)

745/745 [==============================] - 2s 2ms/step - loss: 0.6931 - binary_crossentropy: 0.6931 - accuracy: 0.3941


[0.6931460499763489, 0.6931461095809937, 0.3940539062023163]

In [30]:
model2a.save('/kaggle/working/model_design2.ker')
model2a.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 875)]             0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                56064     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               33280     
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)              

In [31]:
testf_df = pd.read_csv('../input/lish-moa/test_features.csv',index_col='sig_id')

In [32]:
testf_df['cp_type']=testf_df['cp_type'].astype('category')
testf_df['cp_time']=scaler.transform((np.array(testf_df['cp_time'])).reshape(-1,1))
testf_df['cp_dose']=testf_df['cp_dose'].astype('category')
testf_df.loc[:,'g-0':'c-99']=transformer.transform(testf_df.loc[:,'g-0':'c-99'])
testf_df = oenc.transform(testf_df)
testf_df['cp_type']=testf_df['cp_type']-1
testf_df['cp_dose']=testf_df['cp_dose']-1
testf_arr = np.array(testf_df)

In [33]:
tts_pred2 = model2a.predict(testf_arr)
sub_df = pd.DataFrame(tts_pred2,index=testf_df.index,columns=tts_df.columns)
sub_df.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_0004d9e33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id_001897cda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id_002429b5b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id_00276f245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id_0027f1083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
sub_df.to_csv('/kaggle/working/submission.csv')